# Основные этапы работы:

## Этап 0. Установка и настройка оболочки для работы с языком Python

В лабораторной работе использовалсь среда разработки VS Code с расширением Juputer <u>version: 2024.2.0</u>.

Загрузка необходимых библиотек для выполнения лаборатрных работ:

In [1]:
from sklearn import model_selection
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
import random
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.utils import to_categorical

В лабораторной работе используется модели обучения Random Forest, которые предоствалены в библиотеке sklearn. 

Scikit-learn (sklearn) — это один из наиболее широко используемых пакетов Python для Data Science и Machine Learning.

Random Forest (случайный лес) – это алгоритм машинного обучения, который используется для решения задач классификации и регрессии. Он является расширением алгоритма решающих деревьев, который использует ансамбль деревьев для улучшения качества классификации или регрессии. Суть алгоритма заключается в том, что он создает множество решающих деревьев и использует их для предсказания классов объектов. Каждое дерево строится на случайном подмножестве обучающих данных и случайном подмножестве признаков. В результате, каждое дерево в ансамбле получается немного разным, что позволяет уменьшить эффект переобучения и повысить качество предсказаний.

Для работы с генетическими алгоритмами используется каркас DEAP – мощным и  гибким каркасом эволюционных вычислений для решения практических задач с помощью генетических алгоритмов. DEAP (сокращение от Distributed Evolutionary Algorithms in Python – распределенные эволюционные алгоритмы на Python) поддерживает быструю разработку решений с применением генетических алгоритмов и других методов эволюционных вычислений. DEAP предлагает различные структуры данных и инструменты, необходимые для реализации самых разных решений на основе генетических алгоритмов.

# Этап 1. Построение бинарного классификатора.
**Целью этапа:** является создание бинарного классификатора отзывов к фильмам из наборы данных IMDB, использовав для улучшения качества модели генетические алгоритмы. 

**Формулировка задания:** с помощью генетического алгоритма улучшить качество модели машинного обучения с учителем RandomForestClassifier за счёт настройки его гиперпараметров; классифицировать отзывы к фильмам на положительные и отрицательные отзывы, опираясь на текст отзывов, использовав классификатор RandomForestClassifier.

Загрузка набора данных IMDB:

In [2]:
from keras.datasets import imdb

База данных состоит из 50000 отзывов к кинолентам в интернет-базе (Internet Movie Database). Набор разбит на 25000 обучающих и 25 000 контрольных отзывов, каждый набор на 50 % состоит из отрицательных и на 50 % из положительных отзывов. Набор данных IMDB поставляется в составе Keras. Набор готов к использованию: отзывы
(последовательности слов) преобразованы в последовательности целых чисел, каждое из которых определяет позицию слова в словаре.	
	
Подготовим константы и диапазоны значений для работы с генетическим алгоритмом: 

В модели _RandomForestClassifier_ существует множество гиперпараметров, в лабораторной работе мы используем следющие гиперпараметры:
1. n_estimators – количество деревьев в лесу.
2. criterion - функция для измерения качества разделения. Поддерживаемые критерии: gini для примеси Джини и ”log_loss“ и "entropy" для усиления информации по Шеннону.
3. max_depth - максимальная глубина дерева. Если None, то узлы расширяются до тех пор, пока все листья не станут чистыми или пока все листья не будут содержать меньше выборок min_samples_split.
4. min_samples_split - минимальное число выборок, для разделения внутреннего узла.
5. min_samples_leaf - минимальное число образцов должны быть на листе. Точка разделения на любой глубине будет рассматриваться только в том случае, если она оставляет не менее min_samples_leaf образцов подготовки в каждой из левой и правой ветвей. Это может привести к сглаживанию модели, особенно при регрессии.
6. max_features - количество функций, которые следует учитывать при поиске наилучшего разделения: если “sqrt”, то max_features=sqrt(n_features), если “log2”, то max_features=log2(n_features), если None, то max_features=n_features, где n_features это количество особенностей, наблюдаемых во время подгонки (fit).
7. max_leaf_nodes - выращивайте деревья с помощью max_leaf_nodes по принципу "сначала лучше". Лучшие узлы определяются как относительное уменьшение примесей. Если None, то неограниченное количество конечных узлов.
8. bootstrap - используются ли выборки bootstrap при построении деревьев. Если значение равно False, для построения каждого дерева используется весь набор данных.
9. class_weight - веса, связанные с классами в форме {class_label: weight}. Если None, предполагается, что все классы имеют вес один. Для задач с несколькими выводами список dicts может быть предоставлен в том же порядке, что и столбцы y. Режим “balanced” использует значения y для автоматической настройки весов, обратно пропорциональных частотам классов во входных данных, как n_samples / (n_classes * np.bincount(y)). Режим “balanced_subsample” такой же, как и “balanced”, за исключением того, что веса вычисляются на основе начальной выборки для каждого выращенного дерева.
10. ccp_alpha - параметр сложности, используемый для сокращения сложности с минимальными затратами. Будет выбрано поддерево с наибольшей сложностью затрат, которое меньше ccp_alpha. По умолчанию обрезка не выполняется.

In [3]:
# список гиперпараметрв и их допустимы значения:
# n_estimators: int, 
# criterion: string, 
# max_depth: int_None, 
# min_samples_split: int,
# min_samples_leaf: int,
# max_features: string_None, 
# max_leaf_nodes: int_None,
# bootstrap: bool, 
# class_weight: string_None, 
# ccp_alpha: non-negative float
BOUNDS_LOW =  [10, 0, 0, 2, 2, 0, 2, 0, 0, 0.0]
BOUNDS_HIGH = [200, 2, 20, 10, 10, 2, 10, 1, 2, 0.05]
NUM_OF_PARAMS = len(BOUNDS_HIGH)

# Константы генетического алгоритма:
POPULATION_SIZE = 20
P_CROSSOVER = 0.9
P_MUTATION = 0.2 
MAX_GENERATIONS = 10
HALL_OF_FAME_SIZE = 5
CROWDING_FACTOR = 20.0

_BOUNDS_LOW_ — массив содержащий нижние границы значений гиперпараметров.

_BOUNDS_HIGH_ — массив содержащий верхние границы значений гиперпараметров.

Порядок гиперпараметров в массивах _BOUNDS_LOW_ и _BOUNDS_HIGH_:

_[n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, class_weight, ccp_alpha]_.

_POPULATION_SIZE_ — количество индивидуумов в популяции

_P_CROSSOVER_ = 0.9 — вероятность скрещивания

_P_MUTATION_ = 0.2 — вероятность мутации индивидуума

_MAX_GENERATIONS_ = 10 — максимальное количество поколений

_HALL_OF_FAME_SIZE_ — количество индивидуумов, которых мы хотим хранить в зале славы.

_CROWDING_FACTOR_ = 20.0 — фактор вытеснения для скрещивания и мутации.

Подготовим класс для получениия оценки верности классификатора обученной на данных из IMDB: 

Напиишем класс _HyperparameterTuningGenetic_ для оценки верности классификатора:

Метод класса _initIMDBDataset_ выгружает данные из набора данных imdb и вызывает метод _Vectorize_sequences_ для векторизации тренировочных и тестовых данных, также векторизует метки. Аргумент _num_words=10000_ означает, что в обучающих данных будет сохранено только 10000 слов, наиболее часто встречающихся в обучающем наборе отзывов, остальные слова будут отброшены.

Переменные _train_data_ и _test_data_ — это списки отзывов; каждый отзыв — это список индексов слов (кодированное представление последовательности слов).

Переменные _train_labels_ и _test_labels_ — это списки нулей и единиц, где нули соответствуют отрицательным отзывам, а единицы — положительным.

Метод класса _Vectorize_sequences_ принимает список отзывов и выполняет над ним прямое кодирование списков в векторы нулей и единиц и возвращает 10000-мерный вектор. Работа этой функции более подробно описан в отчете первой лабораторной работы.

Метод класса _convertParam_ принимает список params, содержащий значения гиперпараметров типа float, и возвращает преобразованные гиперпараметры в истинных значениях.

Метод класса _getAccuracy_ принимает список чисел типа float, представляющих значения гиперпараметров, вызывает метод convertParam() для преобразования их в истинные значения и инициализирует классификатор _RandomForestClassifier_ с этими значениями. Затем он вычисляет верность классификатора.

Метод класса _formatParams_ принимает список params, вызывает метод _convertParam_ для преобразования их в истинные значения, и возвращает строку, которая показывает значения гиперпараметров более информативно.

In [4]:
class HyperparameterTuningGenetic:

    def __init__(self):
        self.initIMDBDataset()
        
    def initIMDBDataset(self):
        (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
        self.x_train = self.Vectorize_sequences(train_data)
        self.x_test = self.Vectorize_sequences(test_data)
        self.y_train = numpy.asarray(train_labels).astype('float32')
        self.y_test = numpy.asarray(test_labels).astype('float32')
    
    def Vectorize_sequences(self, sequences, dimension=10000):
        results = numpy.zeros((len(sequences), dimension))
        for i, sequence in enumerate(sequences):
            results[i, sequence] = 1.
        return results
    
    def convertParams(self, params):
        n_estimators = round(params[0])
        criterion = ['gini', 'entropy', 'log_loss'][round(params[1])]
        max_depth = None if round(params[2]) == 0 else round(params[2])
        min_samples_split = round(params[3])
        min_samples_leaf = round(params[4])
        max_features = ['sqrt', 'log2', None][round(params[5])]
        max_leaf_nodes = None if round(params[6]) == 0 else round(params[6])
        bootstrap = True if round(params[7]) == 1 else False
        class_weight = ['balanced', 'balanced_subsample', None][round(params[8])]
        ccp_alpha = params[9]
        return n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, class_weight, ccp_alpha

    def getAccuracy(self, params):
        n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, class_weight, ccp_alpha = self.convertParams(params)
        model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split = min_samples_split, min_samples_leaf=min_samples_leaf,
                                       max_features=max_features, max_leaf_nodes=max_leaf_nodes, bootstrap=bootstrap, class_weight=class_weight, ccp_alpha=ccp_alpha)
        model.fit(self.x_train, self.y_train)
        y_pred = model.predict(self.x_test)
        return accuracy_score(self.y_test, y_pred)
    
    def formatParams(self, params):
        return "'n_estimators'=%d, 'criterion'=%s, 'max_depth'=%r, 'min_samples_split'=%d, 'min_samples_leaf'=%d, 'max_features'=%s, 'max_leaf_nodes'=%r, 'bootstrap'=%r, 'class_weight'=%s, 'ccp_alpha'=%.5f" % (self.convertParams(params))

Каркас DEAP включает несколько встроенных эволюционных алгоритмов, находящихся в  модуле _algorithms_. Один из них, _eaSimple_, реализует общую структуру генетического алгоритма и  может заменить большую часть написанного нами кода. Для сбора и печати статистики можно использовать другие полезные объекты _DEAP_, _Statistics_ и _logbook_. У встроенного метода _algorithms.eaSimple_ есть еще одна возможность – зал славы (hall of fame, сокращенно hof). Класс _HallOfFame_, находящийся в модуле _tools_, позволяет сохранить лучших индивидуумов, встретившихся в процессе эволюции, даже если вследствие отбора, скрещивания и мутации они были в какой-то момент утрачены. Зал славы поддерживается в отсортированном состоянии, так что первым элементом всегда является индивидуум с наилучшим встретившимся значением приспособленности.

Однако с помощью _eaSimple_, нам не удается сохранить лучшие решения. Поскольку средняя приспособленность популяции в генетическом алгоритме, возрастает от поколения к поколению, в любой момент может случиться так, что лучшие индивидуумы в текущем поколении исчезнут. Это связано с тем, что операторы отбора, скрещивания и мутации изменяют индивидуумов в процессе создания следующего поколения. Во многих случаях потеря временная, поскольку эти (или даже лучшие) индивидуумы снова появятся в будущем поколении. Но если мы хотим гарантировать, что лучшие индивидуумы обязательно переходят в  следующее поколение, то можем применить факультативную стратегию элитизма. Это означает, что n лучших индивидуумов (_n_  – небольшое, заранее заданное число) копируются в следующее поколение, до того как все места будут заняты потомками, полученными в результате отбора, скрещивания и мутации. Скопированные элитные индивидуумы попрежнему могут использоваться как родители новых индивидуумов. Иногда элитизм оказывает заметный положительный эффект на качество алгоритма, поскольку не нужно тратить времени на повторное открытие хороших решений, потерянных в результате эволюции.

Элитизм позволяет сохранить лучшие решения, защитив их от применения операторов отбора, скрещивания и мутации. Для его реализации придется залезть под капот и модифицировать код алгоритма _DEAP algorithms.eaSimple_, поскольку каркас не дает прямого способа обойти эти операторы.

Метод _eaSimpleWithElitism_ аналогичен оригинальному методу _eaSimple_, но теперь объект _halloffame_ используется для реализации механизма элитизма. Индивидуумы, хранящиеся в объекте _halloffame_, просто копируются в  следующее поколение, не подвергаясь воздействию операторов отбора, скрещивания и мутации. Для этого нужно внести следующие модификации:
* вместо того чтобы отбирать индивидуумов в количестве, равном размеру популяции, мы отбираем их меньше на столько, сколько индивидуумов находится в зале славы:

_offspring = toolbox.select(population, len(population) - hof_size)_

* после применения генетических операторов индивидуумы добавляются из зала славы в популяцию:

offspring.extend(halloffame.items)


Метод _eaSimpleWithElitism_ предполагает, что в toolbox уже зарегистрированы операторы evaluate, select, mate и mutate. Условие остановки задается с помощью параметра ngen – максимального количества поколений.

Метод _eaSimpleWithElitism_ возвращает два объекта – конечную популяцию и объект _logbook_, содержащий собранную статистику. Интересующую насс татистику можно извлечь методом _select()_ и использовать для построения графиков

In [5]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__): # элитарность
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    # Этот алгоритм аналогичен алгоритму Deeper Simple() с той модификацией, 
    # что halloffame используется для реализации механизма элитарности. 
    # Особи, содержащиеся в halloffame, напрямую передаются следующему поколению и 
    # не подвергаются генетическим операторам отбора, скрещивания и мутации
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Оценивайте "individuals" с "fitness"
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None: # параметр halloffame не должен быть пустым!
        raise ValueError("halloffame parameter must not be empty!") 

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Начните процесс смены поколений
    for gen in range(1, ngen + 1):

        # Выберите людей следующего поколения
        offspring = toolbox.select(population, len(population) - hof_size)

        # Расширяйте круг "individuals"
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Оценивайте "individuals" с "fitness"
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # добавляйте лучших обратно в популяцию:
        offspring.extend(halloffame.items)

        # Обновите зал славы сгенерированными "individuals"
        halloffame.update(offspring)

        # Замените текущую популяцию потомством
        population[:] = offspring

        # Добавьте статистику текущей генерации в журнал регистрации
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

Приступим к написанию генетического алгоритма:

In [ ]:
# Создаем экземпляр класса HyperparameterTuningGenetic, который позволит нам проверить различные комбинации гиперпараметров:
test = HyperparameterTuningGenetic()

Здесь мы используем механизм, предлагаемый каркасом DEAP, – класс base.Toolbox. Он используется как контейнер для функций (или операторов) и позволяет создавать новые операторы путем назначения псевдонимов или настройки существующих функций.

In [ ]:
toolbox = base.Toolbox()

Модуль _DEAP_ creator спользуется как метафабрика и позволяет расширять существующие классы, добавляя в них новые атрибуты.

При работе с _DEAP_ значения приспособленности инкапсулированы в классе _Fitness_. _DEAP_ позволяет распределять приспособленность по нескольким компонентам (называемым целями), у каждого из которых есть свой вес. Комбинация весов определяет поведение, или стратегию приспособления в конкретной задаче.

Для определения стратегии в состав _DEAP_ входит абстрактный класс _base.Fitness_, который содержит кортеж _weights_. Этому кортежу необходимо присвоить значения, чтобы определить стратегию и сделать класс пригодным для использования. Для этого мы расширяем базовый класс _Fitness_ с помощью модуля creator. Поскольку мы стремимся максимизировать верность классификатора, определим единственную цель – максимизирующую стратегию приспособления _FitnessMax_:

In [ ]:
# определите единую цель, максимизирующую фитнес-стратегию:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

Второе применение модуля creator – определение индивидуумов, образующих популяцию в генетическом алгоритме. В DEAP класс Individual создается путем расширения базового класса, представляющего хромосому. Кроме того, каждый экземпляр класса Individual должен содержать функцию приспособленности в качестве атрибута.
Чтобы удовлетворить обоим требованиям, мы воспользуемся модулем creator для создания класса creator.Individual:

In [ ]:
# создайте индивидуальный класс на основе списка:
creator.create("Individual", list, fitness=creator.FitnessMax)

Созданный класс Individual расширяет встроенный класс Python list. Это означает, что все хромосомы имеют тип list; в каждом экземпляре класса Individual имеется атрибут fitness созданного ранее класса FitnessMax.

Решение представлено списком чисел типа _float_, принадлежащих различным диапазонам, мы в цикле обойдем все пары, состоящие из нижней и верхней границ, и для каждого гиперпараметра создадим в инструментарии свой оператор, который будет генерировать случайное число в соответствующем диапазоне:

In [ ]:
# определите атрибуты гиперпараметра индивидуально:
for i in range(NUM_OF_PARAMS):
    # "hyperparameter_0", "hyperparameter_1", ...
    toolbox.register("hyperparameter_" + str(i),
                     random.uniform,
                     BOUNDS_LOW[i],
                     BOUNDS_HIGH[i])

Создаем кортеж hyperparameters, содержащий только что созданные генераторы случайных чисел для отдельных гиперпараметров:

In [ ]:
# создайте кортеж, содержащий генератор атрибутов для каждого искомого параметра:
hyperparameters = ()
for i in range(NUM_OF_PARAMS):
    hyperparameters = hyperparameters + \
                      (toolbox.__getattribute__("hyperparameter_" + str(i)),)

Теперь мы можем использовать этот кортеж в сочетании со встроенным в _DEAP_ оператором _initCycle()_, чтобы создать новый оператор _individualCreator_, который инициализирует индивидуум комбинацией случайных значений гиперпараметров:

In [ ]:
# создайте отдельный оператор для заполнения Individual экземпляра:
toolbox.register("individualCreator",# исправить
                 tools.initCycle,
                 creator.Individual,
                 hyperparameters,
                 n=1)

# создайте оператора population для создания списка individual:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

Используем метод _getAccuracy()_ экземпляра _HyperparameterTuningGenetic_ для вычисления приспособленности:

In [ ]:
# fitness calculation
def classificationAccuracy(individual):
    return test.getAccuracy(individual),

toolbox.register("evaluate", classificationAccuracy)

Во многих случаях класс Toolbox используется для настройки существующихфункций из модуля tools, который содержит ряд полезных функций, относящихся к генетическим операциям отбора, скрещивания и мутации, а также утилиты для инициализации.

Определим генетические операторы. В качестве оператора отбора используем турнир размера 2, а для скрещивания и мутации возьмем операторы, специализированные для хромосом в виде списков чисел с плавающей точкой, ограниченных предельными значениями гиперпараметров:

In [6]:
# генетические операторы:
toolbox.register("select", tools.selTournament, tournsize=2) # турнирный отбор = 2
toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,# ограниченный вариант оператора cxSimulatedBinary(), принимающий аргументы low и up – нижнюю и верхнюю границы области поиска соответственно
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)
toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0 / NUM_OF_PARAMS)

Опишем подробнее:
* Имя select зарегистрировано как псевдоним существующей в модуле tools функции selTournament() с аргументом tournsize = 2. В результате создается оператор toolbox.select, который выполняет турнирный отбор с размером турнира 2, т. е. получаем лучшего индивидуума среди случайно выбранных индивидуумов с размером турнира.
* Имя mate зарегистрировано как псевдоним существующей в модуле tools функции _cxSimulatedBinaryBounded_ с аргументами low=BOUNDS_LOW, up=BOUNDS_HIGH и eta=CROWDING_FACTOR, т.е. _cxSimulatedBinaryBounded_ выполняет имитацию двоичного кроссовера, которая изменяет входные элементы на месте. Имитируемый двоичный кроссовер ожидает, что последовательность элементов чисел с плавающей запятой. В данном случае принимает аргументы low и up – нижнюю и верхнюю границы области поиска соответственно, а аргумент eta – степень скученности кроссовера. При высоком eta дети будут похожи на своих родителей, в то время как при небольшом eta решения будут гораздо более отличаться.
* Имя mate зарегистрировано как псевдоним существующей в модуле tools функции _mutPolynomialBounded_. _mutPolynomialBounded_ – полиномиальная мутация, реализованная в оригинальном алгоритме NSGA-II на C Deb. С аргументами low и up, eta тоже самое, а indpb – это независимая вероятность мутации каждого атрибута.

Создаем начальную популяцию оператором populationCreator, задавая размер популяции _POPULATION_SIZE_:

In [ ]:
# создать начальную популяцию (поколение 0):
population = toolbox.populationCreator(n=POPULATION_SIZE)

Для сбора статистики мы воспользуемся классом _tools.Statistics_, предоставляемым _DEAP_. Он позволяет собирать статистику, задавая функцию, применяемую к данным, для которых вычисляется статистика.

Поскольку в нашем случае данными является популяция, зададим функцию, которая извлекает приспособленность каждого индивидуума:

In [ ]:
# подготовьте объект статистики:
stats = tools.Statistics(lambda ind: ind.fitness.values)

Зарегистрируем функции, применяемые к этим значениям на каждом шаге. В нашем примере это функции _NumPy max, min, mean_ и _std_:

In [ ]:
stats.register("max", numpy.max)
stats.register("min", numpy.min)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)

собранная статистика возвращается в объекте logbook в конце работы программы

У метода _eaSimpleWithElitism_ есть еще одна возможность – зал славы (hall of fame, сокращенно hof). Класс HallOfFame, находящийся в модуле tools, позволяет сохранить лучших индивидуумов, встретившихся в процессе эволюции, даже если вследствие отбора, скрещивания и мутации они были в какой-то момент утрачены. Зал славы поддерживается в отсортированном состоянии, так что первым элементом всегда является индивидуум с наилучшим встретившимся значением приспособленности.

По завершении алгоритма атрибут items объекта HallOfFame можно использовать для доступа к списку помещенных в зал славы индивидуумов.

In [8]:
# определите объект зала славы:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

Используем элитистский подход, т. е. без изменения копировать лучших на данный момент индивидуумов из зала славы в следующее поколение:

In [9]:
# выполнение генетического алгоритма с добавленной функцией hof:
population, logbook = eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_GENERATIONS,stats=stats,halloffame=hof,verbose=True)

gen	nevals	max   	min	avg     	std      
0  	20    	0.8022	0.5	0.605098	0.0847468
1  	14    	0.8022	0.5	0.68401 	0.0752981
2  	11    	0.81408	0.5	0.728172	0.0798306
3  	14    	0.81408	0.5	0.769478	0.0729567
4  	11    	0.81408	0.78792	0.804044	0.00520403
5  	14    	0.81408	0.7958 	0.80563 	0.00441642
6  	7     	0.81408	0.7956 	0.805772	0.00520066
7  	15    	0.81636	0.79088	0.806046	0.0071177 
8  	12    	0.81644	0.7518 	0.804574	0.013593  
9  	14    	0.81644	0.79776	0.807442	0.00577423
10 	14    	0.8174 	0.79236	0.806532	0.00819452


После прогона алгоритма для 10 поколений с размером популяции 20 получается результат выше. Она автоматически генерируется методом _eaSimpleWithElitism_ в соответствии с переданным ему объектом статистики, если аргумент verbose равен True.

Печатаем лучшее решение и получем графики изменеия нашей статистики в течении 10 поколений:

In [ ]:
# Наилучшее решение:
print("- Best solution is: ")
print(hof.items[0])
print("params = ", test.formatParams(hof.items[0]))
#print("params = ", "'n_estimators'=%d, 'criterion'=%s, 'max_depth'=%r, 'min_samples_split'=%d, 'min_samples_leaf'=%d, 'max_features'=%s, 'max_leaf_nodes'=%r, 'bootstrap'=%r, 'class_weight'=%s" % test.convertParams(hof.items[0]))
print("Accuracy = %.5f" % hof.items[0].fitness.values[0])

# Извлекаем статистику:
maxFitnessValues, minFitnessValues, meanFitnessValues, stdFitnessValues = logbook.select("max", "min", "avg", "std")

# Построения графиков статистики:
sns.set_style("whitegrid")
plt.plot(maxFitnessValues, color='red', label = 'Max accuracy in generation')
plt.plot(minFitnessValues, color='blue', label = 'Min accuracy in generation')
plt.plot(meanFitnessValues, color='green', label = 'Average accuracy in generation')
#plt.plot(stdFitnessValues, color='black', label = 'Standard deviation accuracy in generation')
plt.legend() 
plt.xlabel('Generation')
plt.ylabel('Max / Min / Average / Standard deviation Fitness')
plt.title('Max, Min, Average and Standard deviation fitness over Generations')
plt.show()

<center><img src="4.1.1.png"></center>
<center>Рис. 1. Статистика в течении 10 поколений</center>

После прогона алгоритма для 10 поколений с размером популяции 20 получаем лучший результат с точностью равной 0.81740. С гиперпараметрами классификатора RandomForestClassifier:
_n_estimators = 131, criterion = entropy, max_depth = 4, min_samples_split = 3, min_samples_leaf = 5, max_features = sqrt, max_leaf_nodes = 5, bootstrap = True, class_weight = balanced_subsample, ccp_alpha = 0.00020_

# Этап 2. Построение многоклассого классификатора.

**Целью этапа:** является создание многоклассового классификатора новостных лент из набора данных Reuters, использовав для улучшения качества модели генетические алгоритмы. 

**Формулировка задания:** создать сеть для классификации новостных лент агентства Reuters на 46 взаимоисключающих тем, с помощью генетического алгоритма улучшить качество модели машинного обучения с учителем RandomForestClassifier за счёт настройки его гиперпараметров и классифицировать новостные ленты.

Загрузка набора данных Reuters:

In [2]:
from keras.datasets import reuters

По сравнению с предыдущем этапом лабораторной работы классов больше двух, значит эта задача относится к категории задач многоклассовой классификации; и, поскольку каждый экземпляр данных должен быть отнесен только к одному классу, эта задача является примером однозначной многоклассовой классификации. 

Reuters - простой набор данных, широко используемых для классификации текста. Существует 46 разных тем; некоторые темы более широко представлены, некоторые менее, но для каждой из них в обучающем наборе имеется не менее 10 примеров.

Было решено убрать некоторые параметры модели _RandomForestClassifier_, так как из-за их использования точность прогнозирования модели падала до 0.0, соответсвенно были убраны следующие гиперпараметры:
1. max_leaf_nodes,
2. class_weight,
3. ccp_alpha.

Также для улучшения точности и скорости были изменеы нижние и верхние границы допустимых значений слудеющих гиперпараметров:
1. n_estimators: от 100 до 200,
3. max_depth: от 2 до 20,
4. min_samples_split: от 2 до 20,
5. min_samples_leaf: от 2 до 20.

Количество поколений уменьшим до 10, и популяция в каждом поколении тоже уменьшим до 20.

In [3]:
# n_estimators: int, 
# criterion: string, 
# max_depth: int_None, 
# min_samples_split: int,
# min_samples_leaf: int,
# max_features: string_None, 
# bootstrap: bool.

BOUNDS_LOW =  [50, 0, 0, 2, 2, 0, 2, 0, 0, 0.0]
BOUNDS_HIGH = [200, 2, 20, 20, 20, 2, 20, 1, 2, 0.05]
NUM_OF_PARAMS = len(BOUNDS_HIGH)

# Константы генетического алгоритма:
POPULATION_SIZE = 20 
P_CROSSOVER = 0.9  # вероятность пересечения
P_MUTATION = 0.2 # вероятность мутации индивидуума
MAX_GENERATIONS = 10
HALL_OF_FAME_SIZE = 5
CROWDING_FACTOR = 20.0  # фактор вытеснения для скрещивания и мутации

Подготовим класс для получениия оценки верности классификатора обученной на данных из Reuters: 

Напиишем класс _HyperparameterTuningGenetic_ для оценки верности классификатора:

По сравнению с предыдущим этапом были изменены следющие функции: 
Метод класса _initIMDBDataset_ изменился на _initReutersDataset_, он выгружает данные из набора данных _reuters_ и вызывает метод _Vectorize_sequences_ для векторизации тренировочных и сестовых данных, также векторизует метки с помощью прямого кодирования (one-hot encoding). Прямое кодирование широко используется для форматирования категорий и также называется кодированием категорий (categorical encoding). Аргумент _num_words=10000_ означает, что в обучающих данных будет сохранено только 10000 слов, наиболее часто встречающихся в обучающем наборе отзывов, остальные слова будут отброшены.

Переменные _train_data_ и _test_data_ — это списки новостных лент, каждая новость — это список индексов слов (кодированное представление последовательности слов).

Переменные _train_labels_ и _test_labels_ — это списки меток определяющий класс принадлежности новостных лент от 0 до 45.

Метод класса _Vectorize_sequences_ остался неизменным.

Метод класса _convertParam_ как и _formatParams_ изменились только в количестве обработки гиперпараметров.

Метод класса _getAccuracy_ осталя неизменным.

In [4]:
class HyperparameterTuningGenetic:

    def __init__(self):
        self.initReutersDataset()
        # self.kfold = model_selection.KFold(n_splits=self.NUM_FOLDS, random_state=self.randomSeed)
        
    def initReutersDataset(self):
        (train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)
        self.x_train = self.Vectorize_sequences(train_data)
        self.x_test = self.Vectorize_sequences(test_data)
        self.y_train = to_categorical(train_labels)
        self.y_test = to_categorical(test_labels)
    
    def Vectorize_sequences(self, sequences, dimension=10000):
        results = numpy.zeros((len(sequences), dimension))
        for i, sequence in enumerate(sequences):
            results[i, sequence] = 1.
        return results
    
    def convertParams(self, params):
        n_estimators = round(params[0])
        criterion = ['gini', 'entropy', 'log_loss'][round(params[1])]
        max_depth = None if round(params[2]) == 0 else round(params[2])
        min_samples_split = round(params[3])
        min_samples_leaf = round(params[4])
        max_features = ['sqrt', 'log2', None][round(params[5])]
        bootstrap = True if round(params[6]) == 1 else False
        return n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap

    def getAccuracy(self, params):
        n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap = self.convertParams(params)
        model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split = min_samples_split, min_samples_leaf=min_samples_leaf,
                                       max_features=max_features, bootstrap=bootstrap)
        model.fit(self.x_train, self.y_train)
        y_pred = model.predict(self.x_test)
        return accuracy_score(self.y_test, y_pred)
    
    def formatParams(self, params):
        return "'n_estimators'=%d, 'criterion'=%s, 'max_depth'=%r, 'min_samples_split'=%d, 'min_samples_leaf'=%d, 'max_features'=%s, 'bootstrap'=%r" % (self.convertParams(params))

Метод _eaSimpleWithElitism_() был взят из первого этапа лабораторной работы без изменения.
Как и последующий код.

In [5]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__): # элитарность
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    # Этот алгоритм аналогичен алгоритму Deeper Simple() с той модификацией, 
    # что halloffame используется для реализации механизма элитарности. 
    # Особи, содержащиеся в halloffame, напрямую передаются следующему поколению и 
    # не подвергаются генетическим операторам отбора, скрещивания и мутации
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Оценивайте "individuals" с недостаточной "fitness"
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!") 
        # параметр halloffame не должен быть пустым!

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Начните процесс смены поколений
    for gen in range(1, ngen + 1):

        # Выберите людей следующего поколения
        offspring = toolbox.select(population, len(population) - hof_size)

        # Расширяйте круг "individuals"
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Оценивайте "individuals" с недостаточной "fitness"
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # добавляйте лучших обратно в популяцию:
        offspring.extend(halloffame.items)

        # Обновите зал славы сгенерированными "individuals"
        halloffame.update(offspring)

        # Замените текущую популяцию потомством
        population[:] = offspring

        # Добавьте статистику текущей генерации в журнал регистрации
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

In [6]:
test = HyperparameterTuningGenetic()
toolbox = base.Toolbox()
# определите единую цель, максимизирующую фитнес-стратегию:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# создайте индивидуальный класс на основе списка:
creator.create("Individual", list, fitness=creator.FitnessMax)
# определите атрибуты гиперпараметра индивидуально:
for i in range(NUM_OF_PARAMS):
    # "hyperparameter_0", "hyperparameter_1", ...
    toolbox.register("hyperparameter_" + str(i),# расширить
                     random.uniform,
                     BOUNDS_LOW[i],
                     BOUNDS_HIGH[i])
# создайте кортеж, содержащий генератор атрибутов для каждого искомого параметра:
hyperparameters = ()
for i in range(NUM_OF_PARAMS):
    hyperparameters = hyperparameters + \
                      (toolbox.__getattribute__("hyperparameter_" + str(i)),)
# создайте отдельный оператор для заполнения Individual экземпляра:
toolbox.register("individualCreator",# исправить
                 tools.initCycle,
                 creator.Individual,
                 hyperparameters,
                 n=1)
# создайте оператора population для создания списка individual:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
# fitness calculation
def classificationAccuracy(individual):
    return test.getAccuracy(individual),

toolbox.register("evaluate", classificationAccuracy)

# генетические операторы:
toolbox.register("select", tools.selTournament, tournsize=2) # турнирный отбор = 2
toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,# ограниченный вариант оператора cxSimulatedBinary(), принимающий аргументы low и up – нижнюю и верхнюю границы области поиска соответственно
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)
toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0 / NUM_OF_PARAMS)

In [7]:
# создать начальную популяцию (поколение 0):
population = toolbox.populationCreator(n=POPULATION_SIZE)

# подготовьте объект статистики:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", numpy.max)
stats.register("min", numpy.min)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
# определите объект зала славы:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

In [8]:
# выполните выполнение генетического алгоритма с добавленной функцией hof:
population, logbook = eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_GENERATIONS,stats=stats,halloffame=hof,verbose=True)

In [ ]:
# Наилучшее решение:
print("- Best solution is: ")
print(hof.items[0])
print("params = ", test.formatParams(hof.items[0]))
#print("params = ", "'n_estimators'=%d, 'criterion'=%s, 'max_depth'=%r, 'min_samples_split'=%d, 'min_samples_leaf'=%d, 'max_features'=%s, 'max_leaf_nodes'=%r, 'bootstrap'=%r, 'class_weight'=%s" % test.convertParams(hof.items[0]))
print("Accuracy = %.5f" % hof.items[0].fitness.values[0])

In [ ]:
# Извлекаем статистику:
maxFitnessValues, minFitnessValues, meanFitnessValues, stdFitnessValues = logbook.select("max", "min", "avg", "std")

# Построения графиков статистики:
sns.set_style("whitegrid")
plt.plot(maxFitnessValues, color='red', label = 'Max accuracy in generation')
#plt.plot(minFitnessValues, color='blue', label = 'Min accuracy in generation')
#plt.plot(meanFitnessValues, color='green', label = 'Average accuracy in generation')
#plt.plot(stdFitnessValues, color='black', label = 'Standard deviation accuracy in generation')
plt.legend() 
plt.xlabel('Generation')
plt.ylabel('Max Fitness')
plt.title('Max fitness over Generations')
plt.show()

<center><img src="4.2.11.png"></center>
<center>Рис. 2. Статистика в течении 10 поколений</center>
После прогона алгоритма для 10 поколений с размером популяции 10 получаем лучши результат с точностью равной 0.66696. С гиперпараметрами классификатора RandomForestClassifier:
_n_estimators = 93, criterion = log_loss, max_depth = None, min_samples_split = 5, min_samples_leaf = 2, max_features = None, bootstrap = False._

# Этап 3. Построение прогноза на основе регрессионной модели
**Целью этапа:** является создание модели машинного обучения _RandomForestRegressor_, дающую прогноз цен на дома из набора данных Boston Housing, использовав для улучшения качества модели генетические алгоритмы.

**Формулировка задания:** построить регрессионную модель для предсказания медианной цены на дома в пригороде Бостона, с помощью генетического алгоритма улучшить качество модели машинного обучения с учителем _RandomForestRegressor_ за счёт настройки его гиперпараметров и предсказать цены на дома в пригороде Бостона.

Загрузка набора данных Boston Housing:

In [2]:
from keras.datasets import boston_housing

В этом этапе мы попытаемся предсказать медианную цену на дома в пригороде Бостона в середине 1970 года. Для этого используем регрессионную модель _RandomForestRegressor_.
Основное отличие скалярной регрессии от двух предыдущих этапов, заключается в предсказании не дискретной метки, а значения на непрерывной числовой прямой: например, температуры воздуха на завтра по имеющимся метеорологическим данным или предсказание времени завершения программного проекта по его спецификациям.

В модели _RandomForestRegressor_ существует множество гиперпараметров, они почти индетичны по сравнению с моделью _RandomForestClassifier_ в этом этапе мы используем следющие гиперпараметры:
1. n_estimators – количество деревьев в лесу.
2. criterion - Функция для измерения качества разделения. Поддерживаемые критерии: “squared_error” для среднеквадратичной ошибки, которая равна уменьшению дисперсии в качестве критерия выбора объекта и минимизирует потерю L2, используя среднее значение каждого терминального узла, “friedman_mse”, который использует среднеквадратичную ошибку с оценкой улучшения Фридмана для потенциальных разделений, “absolute_error” для средней абсолютной ошибки, которая минимизирует потерю L1, используя медиану каждого терминального узла, и “poisson”, который использует уменьшение Отклонение Пуассона для поиска расколов. Обучение с использованием “absolute_error” происходит значительно медленнее, чем при использовании “squared_error".
3. max_depth - максимальная глубина дерева. Если None, то узлы расширяются до тех пор, пока все листья не станут чистыми или пока все листья не будут содержать меньше выборок min_samples_split.
4. min_samples_split - минимальное число выборок, для разделения внутреннего узла.
5. min_samples_leaf - минимальное число образцов должны быть на листе. Точка разделения на любой глубине будет рассматриваться только в том случае, если она оставляет не менее min_samples_leaf образцов подготовки в каждой из левой и правой ветвей. Это может привести к сглаживанию модели, особенно при регрессии.
6. max_features - количество функций, которые следует учитывать при поиске наилучшего разделения: если “sqrt”, то max_features=sqrt(n_features), если “log2”, то max_features=log2(n_features), если None, то max_features=n_features, где n_features это количество особенностей, наблюдаемых во время подгонки (fit).
7. max_leaf_nodes - выращивайте деревья с помощью max_leaf_nodes по принципу "сначала лучше". Лучшие узлы определяются как относительное уменьшение примесей. Если None, то неограниченное количество конечных узлов.
8. bootstrap - используются ли выборки bootstrap при построении деревьев. Если значение равно False, для построения каждого дерева используется весь набор данных.
9. ccp_alpha - параметр сложности, используемый для сокращения сложности с минимальными затратами. Будет выбрано поддерево с наибольшей сложностью затрат, которое меньше ccp_alpha. По умолчанию обрезка не выполняется.

В процессе тестирования программы было замечено, что модель работает быстрее чем , поэтому было решено изменить границы следующих гиперпараметров:

1. n_estimators: от 100 до 1000,
3. max_depth: от 0 до 100,
4. min_samples_split: от 2 до 20,
5. min_samples_leaf: от 2 до 20.
6. max_leaf_nodes: от 2 до 20.

Также повысим значение популяции в каждом поколении до 100 и количество поколений до 20:

In [3]:
# n_estimators: int, 
# criterion: string, 
# max_depth: int_None, 
# min_samples_split: int,
# min_samples_leaf: int,
# max_features: string_None, 
# max_leaf_nodes: int_None,
# bootstrap: bool, 
# ccp_alpha: non-negative float

BOUNDS_LOW =  [100, 0, 0, 2, 2, 0, 2, 0, 0.0]
BOUNDS_HIGH = [1000, 3, 100, 20, 20, 2, 20, 1, 0.05]

NUM_OF_PARAMS = len(BOUNDS_HIGH)

# Константы генетического алгоритма:
POPULATION_SIZE = 100 
P_CROSSOVER = 0.9  # вероятность пересечения
P_MUTATION = 0.2   # вероятность мутации индивидуума
MAX_GENERATIONS = 20
HALL_OF_FAME_SIZE = 5
CROWDING_FACTOR = 20.0  # фактор вытеснения для скрещивания и мутации

Подготовим класс для получениия оценки верности регресионной модели обученной на данных из _boston_housing_: 

Напиишем класс _HyperparameterTuningGenetic_ для оценки верности регресионной модели:

По сравнению с предыдущим этапом были изменены следющие функции: 
Метод класса _initReutersDataset_ изменился на _initBostonDataset_, он выгружает данные из набора данных _boston_housing_ и нормализует данные так как данные имеют самые разные диапазоны. Суть нормализации состоит в том что для каждого признака во входных данных (столбца в матрице входных данных) из каждого значения вычитается среднее по этому признаку, и разность делится на стандартное отклонение, в результате признак центрируется по нулевому значению и имеет стандартное отклонение, равное единице.


Для предсказания воспользуемся данными, как уровень преступности, ставка местного имущественного налога и т. д. Используемый набор данных, имеет интересное отличие от двух предыдущих этапов. Он содержит относительно немного образцов данных: всего 506, разбитых на 404 обучающих и 102 контрольных образца. И каждый признак во входных данных (например, уровень преступности) имеет свой масштаб. Например, некоторые признаки являются пропорциями и имеют значения между 0 и 1, другие - между 1 и 12 и т.д.

Метод класса _Vectorize_sequences_ был удалён.

Метод класса _convertParam_ как и _formatParams_ изменились только в количестве обработки гиперпараметров.

Метод класса _getAccuracy_ поменял модель на _RandomForestRegressor_.

In [4]:
class HyperparameterTuningGenetic:

    def __init__(self):
        self.initBostonDataset()
        
    def initBostonDataset(self):
        (train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
        mean=train_data.mean(axis=0)
        train_data-=mean
        std=train_data.std(axis=0)
        train_data/=std
        test_data-=mean
        test_data/=std

        self.train_data = train_data
        self.train_targets = train_targets
        self.test_data = test_data
        self.test_targets = test_targets
    
    def convertParams(self, params):
        n_estimators = round(params[0])
        criterion = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'][round(params[1])]
        max_depth = None if round(params[2]) == 0 else round(params[2])
        min_samples_split = round(params[3])
        min_samples_leaf = round(params[4])
        max_features = ['sqrt', 'log2', None][round(params[5])]
        max_leaf_nodes = None if round(params[6]) == 0 else round(params[6])
        bootstrap = True if round(params[7]) == 1 else False
        ccp_alpha = params[8]
        return n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, ccp_alpha

    def getAccuracy(self, params):
        n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, ccp_alpha = self.convertParams(params)
        model = RandomForestRegressor(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split = min_samples_split, min_samples_leaf=min_samples_leaf,
                                       max_features=max_features, max_leaf_nodes=max_leaf_nodes, bootstrap=bootstrap, ccp_alpha=ccp_alpha)
        
        model.fit(self.train_data, self.train_targets)
        prediction = model.predict(self.test_data)
        return mean_absolute_error(self.test_targets, prediction)
    
    def formatParams(self, params):
        return "'n_estimators'=%d, 'criterion'=%s, 'max_depth'=%r, 'min_samples_split'=%d, 'min_samples_leaf'=%d, 'max_features'=%s, 'max_leaf_nodes'=%r, 'bootstrap'=%r, 'ccp_alpha'=%.5f" % (self.convertParams(params))

Метод _eaSimpleWithElitism_() был взят из первого этапа лабораторной работы без изменения.

In [5]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__): # элитарность
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    # Этот алгоритм аналогичен алгоритму Deeper Simple() с той модификацией, 
    # что halloffame используется для реализации механизма элитарности. 
    # Особи, содержащиеся в halloffame, напрямую передаются следующему поколению и 
    # не подвергаются генетическим операторам отбора, скрещивания и мутации
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Оценивайте "individuals" с "fitness"
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!") 
        # параметр halloffame не должен быть пустым!

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Начните процесс смены поколений
    for gen in range(1, ngen + 1):

        # Выберите людей следующего поколения
        offspring = toolbox.select(population, len(population) - hof_size)

        # Расширяйте круг "individuals"
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Оценивайте "individuals" с "fitness"
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # добавляйте лучших обратно в популяцию:
        offspring.extend(halloffame.items)

        # Обновите зал славы сгенерированными "individuals"
        halloffame.update(offspring)

        # Замените текущую популяцию потомством
        population[:] = offspring

        # Добавьте статистику текущей генерации в журнал регистрации
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

Так как наша цель минимизировать оценку mae (mean absolute error).

mae - средняя абсолютная ошибка. Это абсолютное значение разности предсказанными и целевыми значениями.

Мы должны изменить стратегию: 

Поскольку мы стремимся минимизировать ошибку регрессионной модели, определим единственную цель – минмизирующую стратегию приспособления _FitnessMin_, а веса _weights_ станут отрицательными значениями. 

Следовательно функция приспособленности класса _Individual_ в качестве атрибута станет _FitnessMin_.

Остальное останется без изменения.

In [6]:
test = HyperparameterTuningGenetic()
toolbox = base.Toolbox()
# определите единую цель, максимизирующую фитнес-стратегию:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# создайте индивидуальный класс на основе списка:
creator.create("Individual", list, fitness=creator.FitnessMin)
# определите атрибуты гиперпараметра индивидуально:
for i in range(NUM_OF_PARAMS):
    # "hyperparameter_0", "hyperparameter_1", ...
    toolbox.register("hyperparameter_" + str(i),# расширить
                     random.uniform,
                     BOUNDS_LOW[i],
                     BOUNDS_HIGH[i])
# создайте кортеж, содержащий генератор атрибутов для каждого искомого параметра:
hyperparameters = ()
for i in range(NUM_OF_PARAMS):
    hyperparameters = hyperparameters + \
                      (toolbox.__getattribute__("hyperparameter_" + str(i)),)
# создайте отдельный оператор для заполнения Individual экземпляра:
toolbox.register("individualCreator",# исправить
                 tools.initCycle,
                 creator.Individual,
                 hyperparameters,
                 n=1)
# создайте оператора population для создания списка individual:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
# fitness calculation
def classificationAccuracy(individual):
    return test.getAccuracy(individual),

toolbox.register("evaluate", classificationAccuracy)
# генетические операторы:mutFlipBit

# генетические операторы:
toolbox.register("select", tools.selTournament, tournsize=2) # турнирный отбор = 2
toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,# ограниченный вариант оператора cxSimulatedBinary(), принимающий аргументы low и up – нижнюю и верхнюю границы области поиска соответственно
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)# коэффициент скученности (eta), задействованный в операциях скрещивания и мутации:
# cxSimulatedBinary() – реализация имитации двоичного скрещивания, величина η передается в параметре eta;
toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0 / NUM_OF_PARAMS)

In [7]:
# создать начальную популяцию (поколение 0):
population = toolbox.populationCreator(n=POPULATION_SIZE)

# подготовьте объект статистики:
stats = tools.Statistics(lambda ind: ind.fitness.values) # разобраться
stats.register("min", numpy.min)
stats.register("max", numpy.max)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
# определите объект зала славы:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

In [8]:
# выполните выполнение генетического алгоритма с добавленной функцией hof:
population, logbook = eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_GENERATIONS,stats=stats,halloffame=hof,verbose=True)

gen	nevals	min    	max    	avg    	std     
0  	100   	2.64809	5.93212	3.30172	0.564761
1  	85    	2.61518	4.60677	3.00661	0.257467
2  	89    	2.61518	3.67322	2.92455	0.187918
3  	87    	2.58659	3.28176	2.83231	0.166791
4  	91    	2.58659	3.31195	2.74736	0.148758
5  	85    	2.58659	3.23223	2.68948	0.119033
6  	87    	2.57681	3.0819 	2.637  	0.0660754
7  	90    	2.55632	2.80247	2.6112 	0.0327832
8  	86    	2.55632	2.66249	2.60587	0.0206812
9  	92    	2.55632	2.84633	2.60766	0.0458618
10 	88    	2.55632	2.81063	2.59713	0.0313357
11 	87    	2.55632	2.91076	2.59581	0.0441575
12 	87    	2.55462	2.84654	2.59217	0.0334119
13 	86    	2.54478	2.80444	2.5876 	0.0269018
14 	91    	2.54478	3.07766	2.59947	0.0765501
15 	92    	2.54478	3.1138 	2.59431	0.0563192
16 	89    	2.54478	2.81143	2.59327	0.0347732
17 	91    	2.53743	2.66135	2.587  	0.0207428
18 	86    	2.53743	3.07766	2.59135	0.0624206
19 	91    	2.53743	3.1138 	2.59257	0.0623328
20 	84    	2.53743	2.94387	2.58966	0.0406859


In [9]:
# найдено наилучшее решение для печати:
print("- Best solution is: ")
print(hof.items[0])
print("params = ", test.formatParams(hof.items[0]))
print("MAE = %.10f" % hof.items[0].fitness.values[0])

- Best solution is: 
[208.92015504714593, 0.1616585288592741, 61.079663038591164, 4.002559822044243, 6.955693971347826, 1.5264439394520408, 19.973491323005792, 0.7549652326332608, 0.01909447049869757]
params =  'n_estimators'=209, 'criterion'=squared_error, 'max_depth'=61, 'min_samples_split'=4, 'min_samples_leaf'=7, 'max_features'=None, 'max_leaf_nodes'=20, 'bootstrap'=True, 'ccp_alpha'=0.01909
MAE = 2.5374286287


In [ ]:
# Извлекаем статистику:
minFitnessValues, maxFitnessValues, meanFitnessValues, stdFitnessValues = logbook.select("min", "max", "avg", "std")

# Построения графиков статистики:
sns.set_style("whitegrid")
#plt.plot(maxFitnessValues, color='red', label = 'Max MAE in generation')
plt.plot(minFitnessValues, color='blue', label = 'Min MAE in generation')
#plt.plot(meanFitnessValues, color='green', label = 'Average MAE in generation')
#plt.plot(stdFitnessValues, color='black', label = 'Standard deviation MAE in generation')
plt.legend() 
plt.xlabel('Generation')
plt.ylabel('Min Fitness')
plt.title('Min fitness over Generations')
#plt.ylabel('Max / Min / Average / Standard deviation Fitness')
#plt.title('Max, Min, Average and Standard deviation fitness over Generations')
plt.show()


<center><img src="4.3.11.png"></center>
<center>Рис. 3. Статистика в течении 20 поколений</center>

После прогона алгоритма для 20 поколений с размером популяции 100 получаем лучший результат с ошибкой MAE равной 2.5374286287. С гиперпараметрами модели RandomForestRegressor: _n_estimators = 209, criterion = squared_error, max_depth = 61, min_samples_split = 4, min_samples_leaf = 7, max_features = None, max_leaf_nodes = 20, bootstrap = True, ccp_alpha = 0.01909_.

Попробуем поменять способ обучения регрессионной модели, используя метод перкрестной проверки по K блокам. Подробнее о реализации написано в первой лабораторной работе.


Класс _HyperparameterTuningGenetic_ для оценки верности регресионной модели изменится только в функции _getAccuracy_:
Используем перекрестную проверку с помощью функции _cross_val_score_ и задаём метрику потери: neg_mean_absolute_error . neg_mean_absolute_error - это потеря регрессии с абсолютной ошибкой среднего значения с отрицательным знаком. Как ошибку получаем средее значение трех оценко mae, так как мы разбили выборку на 3 блока.



In [6]:
class HyperparameterTuningGenetic:

    def __init__(self):
        self.initBostonDataset()
        
    def initBostonDataset(self):
        (train_data, train_targets), (test_data, test_targets) = boston_housing.load_data(test_split=0)
        self.X_data = train_data
        self.y_data = train_targets
        mean=self.X_data.mean(axis=0)
        self.X_data-=mean
        std=self.X_data.std(axis=0)
        self.X_data/=std
    
    def convertParams(self, params):
        n_estimators = round(params[0])
        criterion = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'][round(params[1])]
        max_depth = None if round(params[2]) == 0 else round(params[2])
        min_samples_split = round(params[3])
        min_samples_leaf = round(params[4])
        max_features = ['sqrt', 'log2', None][round(params[5])]
        max_leaf_nodes = None if round(params[6]) == 0 else round(params[6])
        bootstrap = True if round(params[7]) == 1 else False
        ccp_alpha = params[8]
        return n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, ccp_alpha

    def getAccuracy(self, params):
        n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, max_leaf_nodes, bootstrap, ccp_alpha = self.convertParams(params)
        model = RandomForestRegressor(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split = min_samples_split, min_samples_leaf=min_samples_leaf,
                                       max_features=max_features, max_leaf_nodes=max_leaf_nodes, bootstrap=bootstrap, ccp_alpha=ccp_alpha)
        scores = cross_val_score(model, self.X_data, self.y_data, cv=3, scoring = 'neg_mean_absolute_error')
        return scores.mean()
    
    def formatParams(self, params):
        return "'n_estimators'=%d, 'criterion'=%s, 'max_depth'=%r, 'min_samples_split'=%d, 'min_samples_leaf'=%d, 'max_features'=%s, 'max_leaf_nodes'=%r, 'bootstrap'=%r, 'ccp_alpha'=%.5f" % (self.convertParams(params))

Так как наша оценка теперь стала отрицательной, то необходимо теперь максимизировать оценку mae.

Мы должны исправить стратегию, и вернуть к виду, которые были использованы в 1 и 2 этапах. Остальное останется без изменения:

In [7]:
test = HyperparameterTuningGenetic()
toolbox = base.Toolbox()
# определите единую цель, максимизирующую фитнес-стратегию:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# создайте индивидуальный класс на основе списка:
creator.create("Individual", list, fitness=creator.FitnessMax)
# определите атрибуты гиперпараметра индивидуально:
for i in range(NUM_OF_PARAMS):
    # "hyperparameter_0", "hyperparameter_1", ...
    toolbox.register("hyperparameter_" + str(i),# расширить
                     random.uniform,
                     BOUNDS_LOW[i],
                     BOUNDS_HIGH[i])
# создайте кортеж, содержащий генератор атрибутов для каждого искомого параметра:
hyperparameters = ()
for i in range(NUM_OF_PARAMS):
    hyperparameters = hyperparameters + \
                      (toolbox.__getattribute__("hyperparameter_" + str(i)),)
# создайте отдельный оператор для заполнения Individual экземпляра:
toolbox.register("individualCreator",# исправить
                 tools.initCycle,
                 creator.Individual,
                 hyperparameters,
                 n=1)
# создайте оператора population для создания списка individual:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
# fitness calculation
def classificationAccuracy(individual):
    return test.getAccuracy(individual),

toolbox.register("evaluate", classificationAccuracy)
# генетические операторы:mutFlipBit

# генетические операторы:
toolbox.register("select", tools.selTournament, tournsize=2) # турнирный отбор = 2
toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,# ограниченный вариант оператора cxSimulatedBinary(), принимающий аргументы low и up – нижнюю и верхнюю границы области поиска соответственно
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)# коэффициент скученности (eta), задействованный в операциях скрещивания и мутации:
# cxSimulatedBinary() – реализация имитации двоичного скрещивания, величина η передается в параметре eta;
toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0 / NUM_OF_PARAMS)
# создать начальную популяцию (поколение 0):
population = toolbox.populationCreator(n=POPULATION_SIZE)

# подготовьте объект статистики:
stats = tools.Statistics(lambda ind: ind.fitness.values) # разобраться
stats.register("max", numpy.max)
stats.register("min", numpy.min)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
# определите объект зала славы:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

In [8]:
# выполните выполнение генетического алгоритма с добавленной функцией hof:
population, logbook = eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_GENERATIONS,stats=stats,halloffame=hof,verbose=True)

gen	nevals	max     	min     	avg     	std     
0  	100   	-2.56994	-4.62699	-3.09109	0.313142
1  	90    	-2.56184	-3.44418	-2.92395	0.197716
2  	88    	-2.51709	-3.39014	-2.81907	0.178078
3  	87    	-2.50589	-3.0636 	-2.75694	0.156476
4  	84    	-2.50173	-3.1285 	-2.70961	0.16967 
5  	90    	-2.47495	-3.14807	-2.62576	0.140403
6  	84    	-2.47495	-3.17613	-2.56338	0.0932646
7  	75    	-2.45004	-2.72383	-2.53161	0.0452782
8  	86    	-2.45004	-2.71675	-2.50746	0.0356451
9  	93    	-2.42566	-2.54845	-2.49104	0.0238416
10 	85    	-2.42566	-2.66545	-2.48051	0.0303142
11 	84    	-2.42361	-2.51689	-2.46524	0.0194465
12 	82    	-2.41388	-2.65667	-2.45822	0.0278792
13 	90    	-2.41388	-2.52524	-2.44708	0.0147561
14 	91    	-2.41388	-2.63223	-2.4419 	0.029183 
15 	85    	-2.41388	-2.63825	-2.43671	0.0218377
16 	91    	-2.41388	-2.64046	-2.43759	0.02197  
17 	88    	-2.41388	-2.4573 	-2.43285	0.00734367
18 	85    	-2.41388	-2.65198	-2.43618	0.0235556 
19 	92    	-2.41388	-3.14494	-2.44459	0.07401

In [12]:
# найдено наилучшее решение для печати:
print("- Best solution is: ")
print(hof.items[0])
print("params = ", test.formatParams(hof.items[0]))
print("MAE = %.10f" % hof.items[0].fitness.values[0])

- Best solution is: 
[499.60343835210404, 2.379211238128562, 23.322656699802153, 11.853219665696205, 2.088117760953659, 1.8322115001865766, 18.511106619897966, 0.7366816311420722, 0.001161024479828595]
params =  'n_estimators'=500, 'criterion'=friedman_mse, 'max_depth'=23, 'min_samples_split'=12, 'min_samples_leaf'=2, 'max_features'=None, 'max_leaf_nodes'=19, 'bootstrap'=True, 'ccp_alpha'=0.00116
MAE = -2.4138813583


In [ ]:
# Извлекаем статистику:
maxFitnessValues, minFitnessValues, meanFitnessValues, stdFitnessValues = logbook.select("max", "min", "avg", "std")

# Построения графиков статистики:
sns.set_style("whitegrid")
plt.plot(maxFitnessValues, color='red', label = 'Max accuracy in generation')
#plt.plot(minFitnessValues, color='blue', label = 'Min accuracy in generation')
#plt.plot(meanFitnessValues, color='green', label = 'Average accuracy in generation')
#plt.plot(stdFitnessValues, color='black', label = 'Standard deviation accuracy in generation')
plt.legend() 
plt.xlabel('Generation')
plt.ylabel('Max Fitness')
plt.title('Max fitness over Generations')
plt.show()

<center><img src="4.3.21.png"></center>
<center>Рис. 4. Статистика в течении 20 поколений</center>

После прогона алгоритма для 20 поколений с размером популяции 100 получаем лучший результат с ошибкой MAE равной -2.4138813583. С гиперпараметрами модели RandomForestRegressor:
_n_estimators = 500, criterion = friedman_mse, max_depth = 23, min_samples_split = 12, min_samples_leaf = 2, max_features = None, max_leaf_nodes = 19, bootstrap = True, ccp_alpha = 0.00116

Средняя оценка ошибки прогнозирования составляет около 2372 долларов США.

<p style="text-align: center;">Заключение</p>

В этой лабораторной работе мы занимались настройкой различных гиперпараметров Random Forest – алгоритма машинного обучения. Использовали каркас DEAP, которая поддерживает разработку решений с применением генетических алгоритмов и других методов эволюционных вычислений, и с помощью него написали генетический алгоритм для наших задач. Познакомились с его различными модулями: _creator_, _register_ и классами: _Fitness_, _Individual_, _Toolbox_. Использовали элитистский подход в функции _eaSimpleWithElitism_ для получения лучших решений, защитив их от применения операторов отбора, скрещивания и мутации.

1. На первом этапе мы классифицировали отзывы к фильмам на положительные и отрицательные опираясь на текст отзывов. Использовали набор данных IMDB с 50000 отзывами. Подготовили данные для передачи в модель с помощью прямого кодирования списков, т.е. закодировали последовательности целых чисел в бинарную матрицу. Написали класс _HyperparameterTuningGenetic_ для оценки верности классификатора. В классе использовали следующие методы: Метод класса _initIMDBDataset_ выгружает данные из набора данных imdb и вызывает метод _Vectorize_sequences_ для векторизации тренировочных и тестовых данных, также векторизует метки. Метод класса _Vectorize_sequences_ принимает список отзывов и выполняет над ним прямое кодирование списков в векторы нулей и единиц и возвращает 10000-мерный вектор. Метод класса _convertParam_ принимает список params, содержащий значения гиперпараметров типа float, и возвращает преобразованные гиперпараметры в истинных значениях. Метод класса _getAccuracy_ принимает список чисел типа float, представляющих значения гиперпараметров, вызывает метод convertParam() для преобразования их в истинные значения и инициализирует классификатор _RandomForestClassifier_ с этими значениями. Затем он вычисляет точность классификатора. Метод класса _formatParams_ принимает список params, вызывает метод _convertParam_ для преобразования их в истинные значения, и возвращает строку, которая показывает значения гиперпараметров более информативно. После прогона алгоритма на 10 поколений с размером популяции 20 получаем лучший результат с точностью равной 81.17%. С гиперпараметрами классификатора _n_estimators = 131, criterion = entropy, max_depth = 4, min_samples_split = 3, min_samples_leaf = 5, max_features = sqrt, max_leaf_nodes = 5, bootstrap = True, class_weight = balanced_subsample, ccp_alpha = 0.00020_. Сравнивая с моделью, которая была построена на первом этапе первой лабораторной работе, это сеть состяла из двух слоев, с 64 нейронами, функции активации скрытых слоев tanh-relu, на выходном слое Dense была использована функция sigmoid, которая на выходе дает скалярное значение в диапозоне от 0 до 1, была использована бинарная перекрестная энтропия, как функция потерь. Скомпелировали и обучили модель сначала на 20 эпохах, но появляется эффект переобучения сети после третьей эпохи, потом переобучили сеть на трех эпохах, в итоге получили модель показывающаяся точность в 88.06% при потерях в 0.29. В итоге, классификатор _RandomForestClassifier_ проиграл в точности на 7%.


2. На втором этапе мы классифицировали новостные ленты по их темам, опираясь на их текст. Использовали набор данных Reuters c 11228 новостей. Подготовили данные для передачи в сеть с помощью прямого кодирования как в предыдущем этапе лабораторной работы. Написали класс _HyperparameterTuningGenetic_ для оценки верности классификатора. По сравнению с предыдущим этапом были изменены следющие функции класса: метод класса _initIMDBDataset_ изменился на _initReutersDataset_, он выгружает данные из набора данных _reuters_ и вызывает метод _Vectorize_sequences_ для векторизации тренировочных и сестовых данных, также векторизует метки с помощью прямого кодирования (one-hot encoding). Метод класса _Vectorize_sequences_ остался неизменным. Метод класса _convertParam_ как и _formatParams_ изменились только в количестве обработки гиперпараметров. Метод класса _getAccuracy_ осталя неизменным. После прогона алгоритма на 10 поколений с размером популяции 20 получаем лучший результат с точностью равной 66.69%. С гиперпараметрами классификатора RandomForestClassifier: _n_estimators = 93, criterion = log_loss, max_depth = None, min_samples_split = 5, min_samples_leaf = 2, max_features = None, bootstrap = False._. Сравнивая с моделью, которая была построена на втором этапе первой лабораторной работе, это сеть состяла из двух слоев, на первом слое было использовано 4 нейрона, на втором слое использовано 32 нейрона, функции активации скрытых слоев tanh-relu, на выходном слое была использована функция softmax для распределения вероятностей определения класса новосных лент, на этом этапе была использована многокатегориальная перекрестная энтропия, как функция потерь, которая определяет расстояние между распределениями вероятностей. Для нахождения оптимального количество эпох обучения было выбрано сначало 30 эпох, однако результаты и графики показали, что этого было недостаточно, потом обучили на 50 эпохах, в итоге после 43 эпохи начилось переобучение модели. Обучив модель на 43 эпохах получили точность в 71%, при потерях 1.41. В итоге, классификатор _RandomForestClassifier_ проиграл в точности на 4.5%.

3. В третьем этапе была решена задача регрессии, она выполняется с применением иных функций потерь, нежели классификация. В этой задаче Использовали набор данных boston_housing, с количеством 506 экземпляров. Обработали данные с помощью нормализации. Написали класс _HyperparameterTuningGenetic_ для получениия ошибки регресионной модели обученной на данных из _boston_housing_. По сравнению с предыдущим этапом были изменены следющие функции: метод класса _initReutersDataset_ изменился на _initBostonDataset_, он выгружает данные из набора данных _boston_housing_ и нормализует данные так как данные имеют самые разные диапазоны. Суть нормализации состоит в том что для каждого признака во входных данных (столбца в матрице входных данных) из каждого значения вычитается среднее по этому признаку, и разность делится на стандартное отклонение, в результате признак центрируется по нулевому значению и имеет стандартное отклонение, равное единице. Метод класса _Vectorize_sequences_ был удалён. Метод класса _convertParam_ как и _formatParams_ изменились только в количестве обработки гиперпараметров. Метод класса _getAccuracy_ поменял модель на _RandomForestRegressor_. Так как наша цель минимизировать оценку mae. Мы изменили стратегию: единственная цель – минмизировать стратегию приспособления _FitnessMin_, а веса _weights_ поставим отрицательными значениями. функция приспособленности класса _Individual_ в качестве атрибута станла _FitnessMin_. Так как в процессе тестирования _RandomForestRegressor_. Получили ошибку в 2537 долларов США, с гиперпараметрами _RandomForestRegressor_: _n_estimators = 209, criterion = squared_error, max_depth = 61, min_samples_split = 4, min_samples_leaf = 7, max_features = None, max_leaf_nodes = 20, bootstrap = True, ccp_alpha = 0.01909_. Попробовали поменять способ обучения регрессионной модели, используя метод перкрестной проверки по K блокам. Для этого в классе _HyperparameterTuningGenetic_ изменили функцию _getAccuracy_, использовали перекрестную проверку с помощью функции _cross_val_score_ и задали метрику потери _mean_absolute_error_. Получили ошибку в 2414 долларов США, что меньше по сравнению с предущим способом обучения, с гиперпараметрами _RandomForestRegressor_: _n_estimators = 500, criterion = friedman_mse, max_depth = 23, min_samples_split = 12, min_samples_leaf = 2, max_features = None, max_leaf_nodes = 19, bootstrap = True, ccp_alpha = 0.00116_. Сравнивая с моделью, которая была построена на втором этапе первой лабораторной работе, это сеть состяла из двух скрытых слоев, с функциями активации tanh и по 64 нейрона и заканчивается одномерным слоем (линейным слоем), не имеющим функции активации. Скомпелировали модель с парматрами: оптимизатор - rmsprop, функция потерь: mse (среднеквадратичная ошибка) и метрику оценки mae (средняя абсолютная ошибка). Из-за малого количества данных boston_housing решили надежно оценить качество модели с помощью метода перекрестной проверки по K блокам. Обучили модель на 500 эпохах, однако при оценке обучения из-за проблем с масштабированием, а также ввиду относительно высокой дисперсии затруднительно увидеть общую тенденцию. Для оптимизации были опущены первые 60 замеров, которые имеют другой масштаб, отличный от масштаба остальной кривой, а также каждая оценка была заменена экспоненциальным скользящим средним по предыдущим оценкам. В результате переобучили модель на 150 эпох, получая среднюю оценку ошибки прогнозирования, которая составляет около 2372 долларов США. В итоге, классификатор _RandomForestRegressor_ проиграл в прогнозировании на 42 доллара США.

<p style="text-align: center;">Список использованной литературы</p>
    1. Шолле Франсуа. Глубокое обучение на Python. - СПб.: Питер, 2018. - 400 с.: ил. - (Серия «Библиотека программиста»).
    2. Вирсански Э. - Генетические алгоритмы на Python / пер. с англ. А. А. Слинкина. – М.: ДМК Пресс, 2020. – 286 с.: ил.
